In [1]:
# Purpose: Test to see if the method chosen to interpolate radiative kernels and CESM output matter

# By: Ty Janoski
# Updated: 02.10.21

In [2]:
# import statments
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import nc_time_axis
import scipy as sp
from cftime import DatetimeNoLeap
import ngl
import itertools
import scipy.integrate as integrate

%matplotlib inline
%config InlineBackend.figure_format = "pdf"
%config Completer.use_jedi = False

In [3]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on january of a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [4]:
qkern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/q.kernel.nc')
ps = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/PS.nc')

In [5]:
upper_levs = np.array((((qkern.hyam * qkern.P0) + (qkern.hybm * ps.PS)).isel(time=0,lat=0,lon=0)/100).isel(lev=slice(None,12)))
lower_levs = np.arange(200,1001,25)

In [7]:
upper_levs

array([  3.64346569,   7.59481965,  14.35663225,  24.61222   ,
        38.26829977,  54.59547974,  72.01245055,  87.82123029,
       103.31712663, 121.54724076, 142.99403876, 168.22507977])

In [32]:
plevs = np.concatenate([upper_levs,lower_levs])
plevs

array([   3.64346569,    7.59481965,   14.35663225,   24.61222   ,
         38.26829977,   54.59547974,   72.01245055,   87.82123029,
        103.31712663,  121.54724076,  142.99403876,  168.22507977,
        200.        ,  225.        ,  250.        ,  275.        ,
        300.        ,  325.        ,  350.        ,  375.        ,
        400.        ,  425.        ,  450.        ,  475.        ,
        500.        ,  525.        ,  550.        ,  575.        ,
        600.        ,  625.        ,  650.        ,  675.        ,
        700.        ,  725.        ,  750.        ,  775.        ,
        800.        ,  825.        ,  850.        ,  875.        ,
        900.        ,  925.        ,  950.        ,  975.        ,
       1000.        ])

In [172]:
ctrl = read_in('ctrl',1,1,'Q').isel(lat=83,lon=142,time=-1)
ctrl['lev'] = ctrl.hyam * ctrl.P0 + ctrl.hybm * ctrl.PS
ctrl.Q[ctrl.lev > ctrl.PS] = np.nan

exp = read_in('4xCO2',1,1,'Q').isel(lat=83,lon=142,time=-1)
exp['lev'] = exp.hyam * exp.P0 + exp.hybm * exp.PS
exp = exp.interp(lev=ctrl.lev)
exp.Q[exp.lev > exp.PS] = np.nan

ctrl2 = read_in('ctrl',1,1,'Q').isel(time=-1)
ctrl2 = ngl.vinth2p(np.log(ctrl2.Q),ctrl2.hyam,ctrl2.hybm,plevs,ctrl2.PS,1,ctrl2.P0/100,1,kxtrp=False)
ctrl2[ctrl2==1e30] = np.nan

exp2 = read_in('4xCO2',1,1,'Q').isel(time=-1)
exp2 = ngl.vinth2p(np.log(exp2.Q),exp2.hyam,exp2.hybm,plevs,exp2.PS,1,exp2.P0/100,1,kxtrp=False)
exp2[exp2==1e30] = np.nan

diff = np.log(exp.Q) - np.log(ctrl.Q)
diff2 = exp2 - ctrl2

fig, ax = plt.subplots(figsize=(4,6))

plt.plot(diff,diff.lev/100)
plt.plot(diff2[:,83,142],plevs)

plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

<Figure size 288x432 with 1 Axes>

In [173]:
# METHOD 1: vertically interpolate to kernel pressure levels

hold = np.zeros((192,288))

# set latitude, longitude, and time
i = 83
j = 142
t = -1

# read in kernels, control, and 4xCO2 output
qkern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/q.kernel.nc').isel(lat=i,lon=j,time=t)
ps_kern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/PS.nc').isel(lat=i,lon=j,time=t).PS
ctrl = read_in('ctrl',1,1,'Q').isel(lat=i,lon=j,time=t)
trop_c = read_in('ctrl',1,1,'TROP_P').isel(lat=i,lon=j,time=t)
exp = read_in('4xCO2',1,1,'Q').isel(lat=i,lon=j,time=t)
trop_e = read_in('4xCO2',1,1,'TROP_P').isel(lat=i,lon=j,time=t)

# move from hybrid sigma coordinates to pressure levels
qkern['lev'] = qkern.hyam * qkern.P0 + qkern.hybm * ps_kern
ctrl['lev'] = ctrl.hyam * ctrl.P0 + ctrl.hybm * ctrl.PS
exp['lev'] = exp.hyam * exp.P0 + exp.hybm * exp.PS

# regrid ctrl and exp to kernel vertical levels
# also, mask values above tropo
ctrl = ctrl.interp(lev = qkern.lev)
exp = exp.interp(lev = qkern.lev)

ctrl = ctrl.where((ctrl.lev < ctrl.PS) & (ctrl.lev > trop_c.TROP_P))
exp = exp.where((exp.lev < exp.PS) & (exp.lev > trop_e.TROP_P))

diff = np.log(exp.Q) - np.log(ctrl.Q)

dp = np.array((qkern.hyai * qkern.P0 + qkern.hybi * ps_kern).diff(dim='ilev'))

FLNT = (diff * np.array(qkern.FLNT) * dp).sum(dim='lev')/-100

In [33]:
t=-1

# METHOD 2: let's use pyngl to regrid to pressure levels
# read in kernels, control, and 4xCO2 output
qkern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/q.kernel.nc').isel(time=t)
ps_kern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/PS.nc').isel(time=t).PS
ctrl = read_in('ctrl',1,1,'Q').isel(time=t)
trop_c = read_in('ctrl',1,1,'TROP_P').isel(time=t)
exp = read_in('4xCO2',1,1,'Q').isel(time=t)
trop_e = read_in('4xCO2',1,1,'TROP_P').isel(time=t)

# plevs = np.linspace(1000,0,100)
# plevs = np.flip(np.array(qkern.hyam * qkern.P0 + qkern.hybm * 101325)/100)
# plevs = np.flip(hold)

FLNT_new = ngl.vinth2p(qkern.FLNS,qkern.hyam,qkern.hybm,plevs,ps_kern,1,qkern.P0/100,1,kxtrp=False)
FLNT_new[FLNT_new==1e30] = np.nan
ctrl_new = ngl.vinth2p(np.log(ctrl.Q),ctrl.hyam,ctrl.hybm,plevs,ctrl.PS,1,ctrl.P0/100,1,kxtrp=False)
ctrl_new[ctrl_new==1e30] = np.nan
exp_new = ngl.vinth2p(np.log(exp.Q),exp.hyam,exp.hybm,plevs,exp.PS,1,exp.P0/100,1,kxtrp=False)
exp_new[exp_new==1e30] = np.nan

plev_3d = np.broadcast_to(np.expand_dims(plevs,axis=[1,2]),exp_new.shape)

ctrl_new = np.where(plev_3d > np.expand_dims(trop_c.TROP_P/100,axis=0),ctrl_new,np.nan)
exp_new = np.where(plev_3d > np.expand_dims(trop_e.TROP_P/100,axis=0),exp_new,np.nan)

diff_new = exp_new - ctrl_new


In [34]:
plevs = np.broadcast_to(np.expand_dims(plevs,axis=[1,2]),diff_new.shape)
midpoints = (plevs[:-1,:,:] + plevs[1:,:,:])/2
bounds = np.concatenate([np.zeros((1,192,288)),midpoints,np.expand_dims(exp.PS,axis=0)/100])

In [35]:
dz = np.diff(bounds,axis=0)

In [197]:
dz[:,0,0]

array([   5.61914267,    5.35658328,    8.50870018,   11.95583376,
         14.99162987,   16.87207539,   16.61287528,   15.65233804,
         16.86300524,   19.83845606,   23.33891951,   18.50298062,
         10.88746011,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.        ,
         10.        ,   10.        ,   10.        ,   10.     

In [36]:
np.trapz(np.ma.masked_invalid(diff * np.array(qkern.FLNT.isel(lat=83,lon=142))),x=diff.lev/100)

NameError: name 'diff' is not defined

In [37]:
FLNT_p = np.nansum(diff_new * FLNT_new * dz,axis=0)*-1

In [38]:
FLNT_p[83,142]

-157.37528506814283

In [201]:
np.trapz(np.ma.masked_invalid(diff_new * FLNT_new),x=plevs[:,0,0],axis=0)[83,142]

613.6781656982228

In [128]:
plevs[:,0,0].shape

(45,)

In [202]:
FLNT_p[83,142]

-613.6878284539363

In [203]:
plt.imshow(FLNT_p,cmap='RdBu')
plt.colorbar()

<Figure size 432x288 with 2 Axes>

In [204]:
FLNT_hy = np.load('/dx05/janoski/d10/Arctic_Research/cesm-LE/temp/FLNT_test_hybsig.npy')

In [183]:
FLNT_hy[83,142]

-647.3784178559258

In [184]:
plt.imshow(FLNT_hy,cmap='RdBu')
plt.colorbar()
plt.show()

<Figure size 432x288 with 2 Axes>

In [186]:
plt.imshow(FLNT_p - FLNT_hy,cmap='RdBu',vmin=-140,vmax=140)
plt.colorbar()
plt.show()

<Figure size 432x288 with 2 Axes>

In [205]:
fig, ax = plt.subplots(figsize=(4,6))

plt.plot(np.array(diff),diff.lev/100)
plt.plot(diff_new[:,83,142],plev_3d[:,0,0])

plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

<Figure size 288x432 with 1 Axes>

In [206]:
fig, ax = plt.subplots(figsize=(4,6))

plt.plot(dp/100/(np.sum(dp/100,axis=0)),diff.lev/100)
plt.plot(dz[:,83,142] / np.sum(dz[:,83,142]),plev_3d[:,0,0])

plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

<Figure size 288x432 with 1 Axes>